In [ ]:
!pip install --quiet category_encoders
!pip install --quiet h2o

     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 164.8MB 84kB/s 


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import joblib

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, classification_report, roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict, StratifiedKFold, learning_curve
import lightgbm as lgb
from lightgbm import LGBMClassifier
from category_encoders.cat_boost import CatBoostEncoder
import torch
import random
import os

import h2o
from h2o.automl import H2OAutoML

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
h2o.init(
    nthreads=-1,   
    max_mem_size='12G'
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp356uheyj
  JVM stdout: /tmp/tmp356uheyj/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp356uheyj/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 3 days
H2O_cluster_name:,H2O_from_python_unknownUser_71hfe8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
train = h2o.import_file("/content/drive/MyDrive/traincate.csv")
test = h2o.import_file("/content/drive/MyDrive/testcate.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
x = train.columns[1:-1]
y = 'correct'

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
aml = H2OAutoML(max_models=10000, seed=42, max_runtime_secs=1800)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_AutoML_20210623_021147,0.951086,0.26446,0.959623,0.137869,0.290649,0.0844769,59142,0.019475,StackedEnsemble
StackedEnsemble_BestOfFamily_AutoML_20210623_021147,0.95023,0.268239,0.958579,0.138096,0.292021,0.0852763,33013,0.017141,StackedEnsemble
XGBoost_2_AutoML_20210623_021147,0.944332,0.334553,0.952989,0.147056,0.314813,0.0991073,9429,0.001364,XGBoost
XGBoost_1_AutoML_20210623_021147,0.935443,0.336508,0.945683,0.160006,0.321304,0.103236,7967,0.001424,XGBoost
XGBoost_grid__1_AutoML_20210623_021147_model_1,0.934214,0.313971,0.94581,0.165107,0.317141,0.100578,64441,0.003325,XGBoost
XGBoost_grid__1_AutoML_20210623_021147_model_2,0.922606,0.418721,0.929236,0.173192,0.356075,0.126789,3073,0.00094,XGBoost
GBM_grid__1_AutoML_20210623_021147_model_1,0.92212,0.363589,0.934996,0.174782,0.338641,0.114678,41307,0.009939,GBM
XGBoost_3_AutoML_20210623_021147,0.898391,0.387908,0.911141,0.202678,0.354732,0.125835,8812,0.001016,XGBoost
DRF_1_AutoML_20210623_021147,0.895524,0.399064,0.907351,0.207945,0.353868,0.125223,8332,0.003273,DRF
DeepLearning_1_AutoML_20210623_021147,0.886898,0.374844,0.890689,0.204846,0.349642,0.12225,4868,0.003166,DeepLearning


In [ ]:
preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
preds

predict,p0,p1
1,0.126101,0.873899
1,0.525882,0.474118
1,0.160857,0.839143
1,0.0312738,0.968726
1,0.0490547,0.950945
0,0.999989,1.14568e-05
0,0.999999,1.28253e-06
0,1,3.51591e-07
1,0.280571,0.719429
0,0.639677,0.360323
